In [1]:
# Imports
import pandas as pd
import sqlite3
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt


%matplotlib inline

In [2]:
# Establish a connection to the SQLite3 database
conn = sqlite3.connect('data\im.db')

In [28]:
# Explore the data structure and primary keys
master_table = pd.read_sql("""
SELECT *
FROM sqlite_master
WHERE type = 'table'
""", conn)
master_table

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [12]:
# Explore each table's contents
print("movie_basics")
display(pd.read_sql("""
SELECT *
FROM movie_basics
LIMIT 5
""", conn))

print("directors")
display(pd.read_sql("""
SELECT *
FROM directors
LIMIT 5
""", conn))

print("known_for")
display(pd.read_sql("""
SELECT *
FROM known_for
LIMIT 5
""", conn))

print("movie_akas")
display(pd.read_sql("""
SELECT *
FROM movie_akas
LIMIT 5
""", conn))

print("movie_ratings")
display(pd.read_sql("""
SELECT *
FROM movie_ratings
LIMIT 5
""", conn))

print("principals")
display(pd.read_sql("""
SELECT *
FROM principals
LIMIT 5
""", conn))

print("writers")
display(pd.read_sql("""
SELECT *
FROM writers
LIMIT 5
""", conn))

movie_basics


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


directors


,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


known_for


...

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


principals


,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


writers


,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087


Notes:
- SELECT * (movie titles, year, runtime, and genres) FROM movie_basics table by connecting with movie_id
- SELECT averagerating FROM movie_ratings

In [16]:
df = pd.read_csv('data/tn.movie_budgets.csv')
df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [19]:
df['worldwide_gross'].value_counts()

worldwide_gross
$0             367
$8,000,000       9
$2,000,000       6
$7,000,000       6
$4,000,000       4
              ... 
$39,498,360      1
$94,875,000      1
$77,593,167      1
$36,299,670      1
$181,041         1
Name: count, Length: 5356, dtype: int64

In [25]:
df.loc[df['domestic_gross'] == '$0']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
194,95,"Dec 31, 2020",Moonfall,"$150,000,000",$0,$0
479,80,"Dec 13, 2017",Bright,"$90,000,000",$0,$0
480,81,"Dec 31, 2019",Army of the Dead,"$90,000,000",$0,$0
535,36,"Feb 21, 2020",Call of the Wild,"$82,000,000",$0,$0
617,18,"Dec 31, 2012",AstÃ©rix et ObÃ©lix: Au service de Sa MajestÃ©,"$77,600,000",$0,"$60,680,125"
...,...,...,...,...,...,...
5761,62,"Dec 31, 2014",Stories of Our Lives,"$15,000",$0,$0
5764,65,"Dec 31, 2007",Tin Can Man,"$12,000",$0,$0
5771,72,"May 19, 2015",Family Motocross,"$10,000",$0,$0
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0


In [26]:
df.loc[df['worldwide_gross'] == '$0']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
194,95,"Dec 31, 2020",Moonfall,"$150,000,000",$0,$0
479,80,"Dec 13, 2017",Bright,"$90,000,000",$0,$0
480,81,"Dec 31, 2019",Army of the Dead,"$90,000,000",$0,$0
535,36,"Feb 21, 2020",Call of the Wild,"$82,000,000",$0,$0
670,71,"Aug 30, 2019",PLAYMOBIL,"$75,000,000",$0,$0
...,...,...,...,...,...,...
5761,62,"Dec 31, 2014",Stories of Our Lives,"$15,000",$0,$0
5764,65,"Dec 31, 2007",Tin Can Man,"$12,000",$0,$0
5771,72,"May 19, 2015",Family Motocross,"$10,000",$0,$0
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0


In [27]:
df.loc[((df['domestic_gross'] != '$0') & (df['worldwide_gross'] == '$0') | ((df['domestic_gross'] == '$0') & (df['worldwide_gross'] != '$0')))]

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
617,18,"Dec 31, 2012",AstÃ©rix et ObÃ©lix: Au service de Sa MajestÃ©,"$77,600,000",$0,"$60,680,125"
619,20,"Jan 22, 2019",Renegades,"$77,500,000",$0,"$1,521,672"
820,21,"Oct 26, 2018",Air Strike,"$65,000,000",$0,"$516,279"
1325,26,"Dec 31, 2012",Foodfight!,"$45,000,000",$0,"$73,706"
1367,68,"Dec 31, 2006",Les BronzÃ©s 3: amis pour la vie,"$42,000,000",$0,"$83,833,602"
...,...,...,...,...,...,...
5590,91,"Mar 24, 2015",Along the Roadside,"$250,000",$0,"$3,234"
5652,53,"Dec 31, 2015",Lumea e a mea,"$168,000",$0,"$29,678"
5661,62,"Dec 31, 2013",Speak No Evil,"$150,000",$0,"$32,927"
5705,6,"Dec 31, 2011",Absentia,"$70,000",$0,"$8,555"


Notes:
- Movies will need to be matched with movie_id while accounting for alternate titles
- Data includes release date (not just year), budget, domestic gross, worldwide gross

Potential routes of analysis:
- ROI per genre (if wanting best return on investment)
- Ratings per genre (if seeking awards)
- ROI by budget (to determine a reasonable budget -- Low: <$5M, Medium: $5-50M, High: >$50M)
- ROI by length of movie (to determine best timeframe -- Short: <90 mins, Medium: 90-120 mins, Long: >120 mins)